In [ ]:
!pip install transformers datasets evaluate accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

BASE_MODEL = "google-bert/bert-base-uncased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 11

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

data_files = {"train": "/content/training_raw_dataset.csv", "test": "/content/testing_raw_dataset.csv"}
ds = load_dataset("csv", data_files=data_files)

def preprocess_function(examples):
    label = examples["labels"]
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Change this to real number
    examples["label"] = float(label)
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=["text", "labels"])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/24182 [00:00<?, ? examples/s]

Map:   0%|          | 0/6315 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="arg-quality-regression",
    push_to_hub=True,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=0,
    report_to="tensorboard",
)

In [ ]:
import torch

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics_for_regression,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,0.027700,0.039755,0.039755,0.145004,-0.004607,0.973555
2,0.021800,0.034220,0.034220,0.135909,0.135280,0.980839
3,0.016900,0.036734,0.036734,0.140901,0.071741,0.978306
4,0.011400,0.040001,0.040001,0.147706,-0.010812,0.975139
5,0.007500,0.043898,0.043898,0.156399,-0.109298,0.970388
6,0.006000,0.046493,0.046493,0.162557,-0.174855,0.966112
7,0.005100,0.042943,0.042943,0.157412,-0.085148,0.972922
8,0.003700,0.044017,0.044017,0.159018,-0.112290,0.971971
9,0.003500,0.041158,0.041158,0.153356,-0.040054,0.975455
10,0.002900,0.041498,0.041498,0.153729,-0.048651,0.974347


TrainOutput(global_step=16632, training_loss=0.010843272970477769, metrics={'train_runtime': 13151.9535, 'train_samples_per_second': 20.225, 'train_steps_per_second': 1.265, 'total_flos': 3.499371927662899e+16, 'train_loss': 0.010843272970477769, 'epoch': 11.0})

In [ ]:
# trainer.eval_dataset=ds["test"]
# trainer.evaluate()

{'eval_loss': 0.034219712018966675,
 'eval_mse': 0.034219712018966675,
 'eval_mae': 0.13590943813323975,
 'eval_r2': 0.13527970310191495,
 'eval_accuracy': 0.9808392715756136,
 'eval_runtime': 99.1484,
 'eval_samples_per_second': 63.692,
 'eval_steps_per_second': 3.984,
 'epoch': 11.0}

In [ ]:
trainer.push_to_hub()